In [610]:
import urllib
import urllib.request
from bs4 import BeautifulSoup
import numpy as np
import re
import pickle
import os
import csv
import sys

csv.field_size_limit(1000000)

1000000

In [589]:
interactions='https://bnf.nice.org.uk/interaction/'
os.chdir('C:\\Users\\Oliver\\Desktop\\uni\\DataSc u MachLearn MSc\\project\\')

In [294]:
with urllib.request.urlopen(interactions) as response:
        webcont=response.read()
soup=BeautifulSoup(webcont)
data=soup.findAll(href=True)
alllinks=list(data)

In [503]:
linklist=[]
for i in range(35,len(data)):
    endlink=str(data[i]).find('html')
    endlink+=4
    if str(data[i])[9]!='a':
        break
    linklist.append(str(data[i])[9:endlink])

In [626]:
ingb=0

if ingb==1:
    alldata=[]
    for i in range(len(linklist)):
        alldata.append(linklist[i][:-7])
        if alldata[-1][0]!='a':
            break
        with urllib.request.urlopen(interactions+linklist[i]) as response:
            currwebcont=response.read()
        currsoup=BeautifulSoup(currwebcont)
        finddivint=currsoup.findAll("div","interactions")
        alldata.append(finddivint)
        currint=[]
        for i in finddivint:
            currint.append(str(i))

        interactants={} #add name, color, text

        stop=0
        currpos=0
        tofind='<h5 class="interactant"><span>'
        tofind2='<div class="interaction-message'
        while stop==0:
            #Name:
            currpos0=currint[0][currpos:].find(tofind)
            if currpos0==-1:
                stop=1
                continue
            else:
                currpos0+=len(tofind)
            currpos1=currint[0][currpos+currpos0:].find('<')
            name=currint[0][currpos+currpos0:currpos+currpos0+currpos1]
            currpos+=currpos0+currpos1

            #Text:
            currposnext=currint[0][currpos:].find(tofind)
            if currposnext==-1:
                currposnext=len(currint[0])+1
            cleaner=re.compile('<.*?>')

            cleantext=""
            cleantext=re.sub(cleaner,'',currint[0][currpos:currpos+currposnext])
            cleantext=cleantext.replace('\r',' ').replace('\n',' ')
            cleantext=' '.join(cleantext.split())


            #Severity/color:
            currseverity=""
            currpos0=currint[0][currpos:].find(tofind2)
            if currpos0!=-1:
                currpos1=currint[0][currpos+currpos0+len(tofind2):].find('"')
                currseverity=currint[0][currpos+currpos0+len(tofind2):currpos+currpos0+len(tofind2)+currpos1]
            interactants[name]=[currseverity.strip(),cleantext]
        alldata.append(interactants)
    with open("interactants.csv",'w', newline='',encoding="utf-8") as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(alldata)
else:
    interactions=[]
    with open('interactants.csv', 'r',newline='',encoding="utf-8") as csvfile:   # Unpickling
        csvreader = csv.reader(csvfile,quoting=csv.QUOTE_ALL)
        for row in csvreader:
            interactions.append(row)
    if len(interactions)==1:
        interactions=interactions[0]
    
    for i in range(0,len(interactions),3):
        alldata.append(interactions[i])
        alldata.append(interactions[i+2])
        
        
 

In [646]:
with open("interactions.csv",'w',newline='', encoding="utf-8") as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(alldata)